In [1]:
import pandas as pd
import numpy as np
import json
import gc
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
import pytorch_lightning as pl
import albumentations
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
import torchvision
from typing import Any, Dict, List, Union, Optional
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import cv2
import torch
from efficientnet_pytorch import EfficientNet
import numpy as np
import timm
from PIL import Image
from PIL import ImageFile

In [2]:
import pytorch_lightning
pytorch_lightning.__version__

'1.1.0'

In [3]:
pytorch_lightning.utilities.seed.seed_everything(seed=42)

42

In [4]:
torch.__version__

'1.7.0'

In [5]:
taylor_B4 = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\0\epoch=13_valid_loss=0.3129_valid_accuracy=0.9002.ckpt',
             1: r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\1\epoch=8_valid_loss=0.3055_valid_accuracy=0.9007.ckpt',
             2: r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\2\epoch=12_valid_loss=0.3124_valid_accuracy=0.8988.ckpt',
             3: r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\3\epoch=11_valid_loss=0.3364_valid_accuracy=0.8909.ckpt',
             4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\4\epoch=9_valid_loss=0.3229_valid_accuracy=0.8916.ckpt',
            }

focalCosine_B4 = {0:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\0\epoch=21_valid_loss=0.1313_valid_accuracy=0.8988.ckpt',
             1: r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\1\epoch=11_valid_loss=0.1244_valid_accuracy=0.9070.ckpt',
             2: r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\2\epoch=7_valid_loss=0.1304_valid_accuracy=0.8967.ckpt',
             3: r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\3\epoch=17_valid_loss=0.1344_valid_accuracy=0.8913.ckpt',
             4:r'C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\4\epoch=13_valid_loss=0.1341_valid_accuracy=0.8946.ckpt',
            }

In [17]:
def runcustomSWA(modelMapping1=taylor_B4,modelMapping2=focalCosine_B4,outDir=r'C:\Users\Kaggle\Leaf_Classification\saved_models\Custom SWA\B4_FocalCosine_B4_Taylor_TopModel_Each_Fold',fold=0):
    
    model1 = modelMapping1[fold]
    model2 = modelMapping2[fold]
    
    print(modelMapping1[fold],print(modelMapping2[fold]))

    swa_state_dict = torch.load(model1, map_location=lambda storage, loc: storage)['state_dict']
    
    for k,v in swa_state_dict.items():
        swa_state_dict[k] = torch.zeros_like(v,dtype=torch.float32)
        
    for wts in [model1,model2]:
        state_dict = torch.load(wts, map_location=lambda storage, loc: storage)['state_dict']
        for k,v in state_dict.items():
            swa_state_dict[k] += v
    print('')

        #----
    for k,v in swa_state_dict.items():
        swa_state_dict[k] /= 2

        
    
    if not os.path.exists(outDir):
        os.makedirs(outDir)
        
    name = outDir+f"\_Avg_fold_{fold}.ckpt"
    torch.save(swa_state_dict, name)

    del swa_state_dict,state_dict,;gc.collect()

In [18]:
for fold in range(5):
#     print(fold)
    runcustomSWA(fold=fold)

C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\0\epoch=21_valid_loss=0.1313_valid_accuracy=0.8988.ckpt
C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\0\epoch=13_valid_loss=0.3129_valid_accuracy=0.9002.ckpt None

C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\1\epoch=11_valid_loss=0.1244_valid_accuracy=0.9070.ckpt
C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mixup_0.3_TaylorLoss_timm_newhead\1\epoch=8_valid_loss=0.3055_valid_accuracy=0.9007.ckpt None

C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_sz_576_Mixup_0.3_focalCosinesLoss_timm_newhead\2\epoch=7_valid_loss=0.1304_valid_accuracy=0.8967.ckpt
C:\Users\Kaggle\Leaf_Classification\saved_models\Image 576\tf_efficientnet_b4_ns_sz_576_Mix